Perfecto. Iniciaré una investigación exhaustiva que cubra los modelos ganadores en la competencia de Kaggle Favorita Grocery Sales Forecasting, así como modelos de vanguardia propuestos en artículos académicos y plataformas como GitHub, sin restricción temporal y en fuentes tanto en español como en inglés. Te entregaré un informe comparativo y técnico con cuatro modelos recomendados al final.

Te avisaré tan pronto como el informe esté listo para tu revisión.


# Forecasting de ventas – Kaggle Corporación Favorita

El desafío *Favorita Grocery Sales Forecasting* consiste en **predecir las ventas unitarias (`unit_sales`) de cada producto en cada tienda para los siguientes 16 días**. El objetivo es anticipar demandas a corto plazo y así optimizar inventarios evitando roturas de stock u exceso de inventario. La métrica de evaluación es el **Error Cuadrático Medio Logarítmico Ponderado Normalizado** (NWRMSLE), definido como la raíz cuadrada del promedio ponderado de las diferencias en escala logarítmica: $\text{NWRMSLE}=\sqrt{\frac{\sum_i w_i(\ln(\hat y_i+1)-\ln(y_i+1))^2}{\sum_i w_i}}\,. $ Aquí los artículos perecederos llevan un peso mayor (1.25 vs 1.0). Esta métrica penaliza por igual errores relativos grandes y pequeños, evitando dar demasiado peso a productos de alto volumen.

El **conjunto de datos** incluye:

* `train.csv`: Historial diario de ventas (`unit_sales`) con columnas *id, date, store\_nbr, item\_nbr, onpromotion*. (Solo filas con ventas > 0).
* `test.csv`: Fechas (a 16 días futuros) de tienda-item combos a predecir, incluye flag `onpromotion`.
* `stores.csv`: Metadatos de tienda (*store\_nbr, city, state, type, cluster*).
* `items.csv`: Metadatos de producto (*item\_nbr, family, class, perishable*).
* `transactions.csv`: N° diario de transacciones por tienda (en periodo de entrenamiento).
* `oil.csv`: Precio diario del petróleo (Ecuador) para fechas de entrenamiento y prueba.
* `holidays_events.csv`: Días feriados y eventos especiales con sus atributos.

Estos archivos permiten enriquecer los datos básicos de ventas con información externa (tráfico, promociones, eventos, etc.). Por ejemplo, el precio del petróleo puede reflejar la economía local; las transacciones totales informan flujo de clientes; los feriados influyen en patrones de consumo.

## Soluciones finalistas

Los equipos mejor posicionados en la competencia usaron principalmente **modelos de árboles potentes** (LightGBM, XGBoost, RandomForest) junto con **redes neuronales temporales** (CNN/seq2seq, RNN, WaveNet) y ensamblajes (blending) de ellos. Por ejemplo, las soluciones top consistieron en promediar múltiples modelos: uno de primer lugar (Equipo *Eureka*) empleó fundamentalmente LightGBM con ingeniería de características rica, mientras que el subcampeón (SoLucky) usó un CNN dilatado estilo *WaveNet*. Otra solución de alto puesto combinó tres modelos: LightGBM, una red *CNN+DNN* y un modelo *seq2seq* con GRU. En general, los enfoques ganadores lograron puntajes NWRMSLE privados del orden de \~0.50–0.51 (el podio finalizó en torno a enero de 2018).

**Modelos destacados:**

* **LightGBM/XGBoost:** frecuentemente en ensamblaje con múltiples semillados. Muy usados por su rapidez y capacidad de manejar features heterogéneas. Un ejemplo (terceros lugares) blend-LGBM obtuvo \~0.515 en LB privado (no seleccionado).
* **Redes convolucionales secuenciales:** conv1D dilatado (inspirado en WaveNet) para captar dependencias temporales.
* **Modelos secuencia a secuencia (RNN):** encoder-decoder con LSTM/GRU, a menudo con atención. Permiten modelar autoregresivamente los 16 días futuros.
* **Modelos probabilísticos:** aunque menos frecuentes, algunos usaron DeepAR (Amazon) o incluso TFT (Temporal Fusion Transformer) para aprendizaje de patrones temporales multivariantes.
* **Ensamblajes:** combinar (ponderar) los distintos tipos de modelo suele mejorar la robustez de la predicción. Por ejemplo, varios kernels públicos (Kaggle) mostraron blends de LightGBM con redes neuronales.

**Ingeniería de características clave:** Las soluciones ganadoras se apoyaron en intensiva generación de features temporales y de agregados externos. Entre las técnicas más comunes están:

* **Lags temporales:** ventas retrasadas (por ejemplo lag 1, 7, 14, 28 días) y medias móviles recientes. Esto captura estacionalidad semanal y tendencias recientes.
* **Agregados grupales:** promedios o totales históricos por tienda, ítem, familia o combinaciones (por día de semana, etc.). Un equipo calculó ventas promedio por *(tienda, día de semana)* o *(ítem, día de semana)* utilizando un estimador bayesiano (con priors de grupos más grandes) para reducir ruido. También generaron características como «promedio por ítem menos promedio general» para reducir correlación entre features.
* **Indicadores temporales y categóricos:** variables dummies de día de la semana, día del mes, feriado, etc. Se usan flags de eventos especiales (p.ej. festividades, eventos nacionales).
* **Promociones:** tratar `onpromotion` fue crítico. Se calcularon efectos de promoción para cada ítem/tienda (p.ej. incremento esperado en log ventas). También se añadieron las variables «días desde última promoción» y «días hasta próxima promoción».
* **Tratamiento de ceros:** como el `train.csv` solo incluye ventas positivas, algunos equipos reinsertaron filas con venta=0 para las combinaciones tienda-ítem que sí existen en tiendas (tras la primera aparición del ítem), asumiendo que no se registraron ventas. De este modo el modelo puede aprender que *no vender* es un resultado posible.
* **Datos externos:** aunque no siempre mejoraron, se incorporaron precio del petróleo, transacciones diarias y características de tiendas (cluster, tipo) e ítems (familia, perecedero). Muchos equipos intentaron usar la serie de petróleo (dada la dependencia petrolera de Ecuador), pero varios informes indican escaso impacto de esta variable.

Ejemplos de notebooks y repositorios incluyen el repositorio *btrotta/kaggle-favorita* (top \~3%) y *LenzDu/Kaggle-Competition-Favorita* (top5), donde se detallan estas estrategias.

## Modelos avanzados alternativos (literatura académica)

Más allá de Kaggle, la literatura reciente propone modelos profusamente basados en aprendizaje profundo para forecasting retail a gran escala. Entre ellos destacan:

* **Transformers y variantes (Autoformer, Informer, TFT, etc.):** Estudios han demostrado que arquitecturas basadas en atención superan ampliamente modelos clásicos (ARIMA, ETS) en series de retail. Por ejemplo, un estudio comparó modelos Transformer, Informer, Autoformer, PatchTST y TFT sobre datos de retail (M5) e informó mejoras de hasta 26–29% en MASE (error absoluto escalado) frente a un naive estacional. En particular, Transformer puro, Informer y TFT lideraron el ranking; Autoformer y PatchTST también superaron métodos tradicionales, aunque requirieron sintonización adicional. Estas arquitecturas destacan en forecast a corto plazo y largo plazo (Autoformer emplea un mecanismo de “auto-correlación” para descomponer señal en tendencia/estacionalidad). Por ejemplo, el **Autoformer** (Zhou et al.) alcanza mejoras del orden del 38% en tareas largas de forecasting.
* **Redes convolucionales temporales (TCN):** Modelos basados en convolución causal (como WaveNet) capturan patrones de dependencia temporal de manera eficiente. En particular, un enfoque híbrido reciente combinó un **TCN** con un componente Transformer para pronóstico de ventas. Este modelo, probado con el dataset Favorita, obtuvo resultados de punta (por ejemplo MAE≈2.01, RMSE≈2.81) superando a LSTM/GRU y hasta al TFT. Esto sugiere que las convoluciones dilatadas son muy efectivas en ventas retail multivariadas.
* **Redes de grafos (GNN):** Las relaciones entre artículos (complementarios o sustitutos) pueden modelarse con **Graph Neural Networks**. Por ejemplo, el método *GraphDeepAR* integra un encoder GNN sobre un grafo de similitud de ítems dentro de un modelo DeepAR probabilístico. En experimentos reales de retail, GraphDeepAR superó consistentemente a DeepAR: obtuvo menor RMSE/MAE promedio en ventas de retail, especialmente mejorando en *cold starts* y artículos “conectados” en el grafo. Esto indica que explotar correlaciones entre artículos similares aumenta la precisión del forecasting. (Sin embargo, las GNN añaden complejidad computacional significativa).
* **Otros modelos contemporáneos:** Se han aplicado variantes como *Crossformer* (modela dependencias cruzadas entre series), *N-BEATS* (red residual profunda interpretable) y diversas redes híbridas. En general, estos métodos suelen requerir GPUs y mayor capacidad de cómputo, pero ofrecen mejoras en escenarios con muchos productos y relaciones complejas.

En síntesis, la investigación académica resalta el poder de los Transformers y las GNN para forecasting retail: mejoran la precisión a costa de requerir más recursos computacionales y diseño de modelos.

## Comparación de modelos

A continuación se resume comparativamente algunos enfoques destacados, considerando precisión, origen, recursos, escalabilidad e implementación:

| **Modelo**                  | **Precisión**                      | **Origen**           | **Recursos necesarios**                    | **Escalabilidad**                            | **Facilidad de uso**                                   |
| --------------------------- | ---------------------------------- | -------------------- | ------------------------------------------ | -------------------------------------------- | ------------------------------------------------------ |
| LightGBM (ensemble)         | NWRMSLE \~0.50–0.52 (líder Kaggle) | Kaggle (competencia) | CPU/cluster (Spark/Dask), memoria moderada | Muy escalable (paralelo por instancias)      | Alto (bibliotecas maduras, fácil de entrenar)          |
| CNN dilatado (WaveNet-like) | NWRMSLE \~0.51 (segundo Kaggle)    | Kaggle (competencia) | GPU (cálculos convolucionales)             | Moderada (depende de minibatches)            | Media (requiere PyTorch/TF, ajuste de hiperparámetros) |
| GraphDeepAR (DeepAR+GNN)    | RMSE \~196 vs 205 (sin GNN)        | Academia             | GPU (GluonTS, MXNet)                       | Media (entrena por lotes, pero grafo grande) | Baja-media (más complejidad de implementación)         |
| Autoformer (Transformer)    | Mejora \~38% sobre modelos previos | Academia             | GPU (atención compleja)                    | Limitada para muy largas series              | Media (existe código, pero modelos nuevos)             |
| TCN+Transformer híbrido     | MAE=2.01, RMSE=2.81 (Favorita)     | Academia             | GPU (Red neuronal)                         | Moderada (depende de secuencia)              | Media (requiere definir arquitectura personalizada)    |
| LSTM/Seq2seq (RNN)          | Peor que CNN en Competencia        | Kaggle/Academia      | GPU/CPU según implementación               | Limitada (difícil paralelismo temporal)      | Media (bien soportado en librerías)                    |

*Nota:* Las cifras de precisión se comparan en métricas reportadas (NWRMSLE o RMSE) según la fuente, y corresponden a resultados de competiciones o estudios.

## Recomendaciones finales

Para el problema de Favorita, se recomiendan cuatro enfoques complementarios:

* **Ensamblaje LightGBM (GBM):** Uno de los más efectivos en la competencia. Fortalezas: muy buena precisión con entrenamiento rápido en CPU (o clústeres Spark/Dask), fácil escalado horizontal, e interpretabilidad parcial (features). Limitaciones: requiere extensa ingeniería de variables (lags, agregados) y validación cuidadosa. Condición óptima: cuando disponemos de datos bien estructurados y recursos de cómputo moderados.
  Ejemplo de configuración LightGBM en Python:

  ```python
  import lightgbm as lgb
  params = {'objective':'regression','metric':'rmse','learning_rate':0.05,'num_leaves':64}
  lgb_train = lgb.Dataset(X_train, label=y_train)
  model = lgb.train(params, lgb_train, num_boost_round=1000, early_stopping_rounds=50, valid_sets=[lgb_train])
  ```

  *Infraestructura:* Clústeres Spark/Dask pueden acelerar la generación de características y el entrenamiento (distribuir datos), sin necesidad de GPU.

* **Modelo CNN secuencial (p.ej. WaveNet-like):** Utiliza convoluciones dilatadas para aprender dependencias temporales. Fue usado con éxito en las soluciones finalistas (2º lugar). Ventajas: modela patrones temporales de largo plazo y no requiere tantas variables manuales. Desventajas: exige GPU para entrenamiento (alta capacidad de memoria) y es más complejo de ajustar. Ideal cuando se dispone de hardware GPU y se desea capturar rápidamente dinámicas temporales complejas. *Infraestructura:* GPU (nube o local) con PyTorch/TensorFlow; orquestación con Airflow/Kubeflow para pipelines de entrenamiento periódico.

* **DeepAR con GNN (GraphDeepAR):** Incorpora relaciones entre productos en un forecast probabilístico. El modelo GraphDeepAR ha mostrado mejoras significativas en precisión frente a DeepAR estándar, especialmente en productos «nuevos» o conectados. Fortalezas: aprovecha características de similaridad entre ítems (información de grafo) y genera pronósticos probabilísticos útiles. Limitaciones: alto coste computacional (entrenamiento más lento por GNN) y requiere diseño de grafo de similitudes. Se recomienda si se dispone de recursos GPU adecuados y se quiere explotar relaciones inter-producto. *Infraestructura:* GPU con framework GluonTS/MXNet o PyTorch; es conveniente usar instancias escalables y planificar entrenamientos largos vía Kubernetes/AWS SageMaker.

* **Autoformer (Transformer):** Arquitectura avanzada de Transformer específicamente para series largas. Captura eficientemente patrones periódicos a largo plazo. Fortalezas: alto poder predictivo (mejor que RNN tradicionales) y capaz de manejar múltiples escalas temporales. Limitaciones: exige GPU con gran memoria (cálculo de atención) y complejidad en arquitectura. Útil si el horizonte de pronóstico es largo o los datos presentan estacionalidades anidadas. *Infraestructura:* GPU (preferiblemente multi-GPU) y bibliotecas de Transformer; orquestar entrenamientos con contenedores Docker y agendamiento (Airflow) para mantener actualizados los modelos.

En todos los casos, se sugiere usar pipelines de datos con **Spark/Dask** para el preprocesamiento de series grandes y feature engineering, y tecnologías de orquestación (**Airflow**, **Kubeflow**) para ejecutar entrenamientos regulares y despliegues. Además, es recomendable el uso de GPUs para acelerar modelos basados en DL (CNN, Transformers, GNN), mientras que los modelos basados en boosting pueden escalar bien en CPUs distribuidas.

**Bibliografía:** Estudio de soluciones finales en Kaggle, literatura técnica sobre métricas y modelos avanzados. Cada enfoque seleccionado combina la precisión lograda con su viabilidad práctica en infraestructura real.


## References

@misc{favorita-kaggle,
  title        = {Corporación Favorita Grocery Sales Forecasting},
  howpublished = {\url{https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting}},
  note         = {Accedido: 09 de mayo de 2025} :contentReference[oaicite:0]{index=0}
}

@article{medium-review-favorita-2017,
  author  = {Ezzat, Sandra},
  title   = {Review: Kaggle Corporación Favorita Grocery Sales Forecasting – Part I},
  journal = {Towards Data Science, Medium},
  year    = {2017},
  url     = {https://medium.com/towards-data-science/review-kaggle-corporaci%C3%B3n-favorita-grocery-sales-forecasting-part-i-9330b7350713},
  note    = {Accedido: 09 de mayo de 2025} :contentReference[oaicite:1]{index=1}
}

@inproceedings{ke2017lightgbm,
  title     = {LightGBM: A Highly Efficient Gradient Boosting Decision Tree},
  author    = {Ke, Guolin and Meng, Qi and Finley, Thomas and Wang, Taifeng and Chen, Wei and Ma, Weidong and Ye, Qi and Liu, Tie-Yan},
  booktitle = {Advances in Neural Information Processing Systems},
  year      = {2017},
  url       = {https://proceedings.neurips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf}
  :contentReference[oaicite:2]{index=2}
}

@article{oord2016wavenet,
  title   = {WaveNet: A Generative Model for Raw Audio},
  author  = {van den Oord, Aaron and Dieleman, Sander and Zen, Heiga and Simonyan, Karen and Vinyals, Oriol and Graves, Alex and Kalchbrenner, Nal and Senior, Andrew and Kavukcuoglu, Koray},
  journal = {arXiv preprint arXiv:1609.03499},
  year    = {2016},
  url     = {https://arxiv.org/abs/1609.03499}
  :contentReference[oaicite:3]{index=3}
}

@article{salinas2017deepar,
  title   = {DeepAR: Probabilistic Forecasting with Autoregressive Recurrent Networks},
  author  = {Salinas, David and Flunkert, Valentin and Gasthaus, Jan and Januschowski, Tim},
  journal = {arXiv preprint arXiv:1704.04110},
  year    = {2017},
  url     = {https://arxiv.org/abs/1704.04110}
  :contentReference[oaicite:4]{index=4}
}

@article{lim2019tft,
  title   = {Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting},
  author  = {Lim, Bryan and Arik, Sercan O and Loeff, Nicolas and Pfister, Tomas},
  journal = {International Journal of Forecasting},
  volume  = {37},
  number  = {4},
  pages   = {1748--1764},
  year    = {2021},
  url     = {https://arxiv.org/abs/1912.09363}
  :contentReference[oaicite:5]{index=5}
}

@article{wu2021autoformer,
  title   = {Autoformer: Decomposition Transformers with Auto-Correlation for Long-Term Series Forecasting},
  author  = {Wu, Haixu and Xu, Jiehui and Wang, Jianmin and Long, Mingsheng},
  journal = {Advances in Neural Information Processing Systems},
  year    = {2021},
  url     = {https://arxiv.org/abs/2106.13008}
  :contentReference[oaicite:6]{index=6}
}

@article{oreshkin2020nbeats,
  title   = {N-BEATS: Neural Basis Expansion Analysis for Interpretable Time Series Forecasting},
  author  = {Oreshkin, Boris N and Carpov, Dmitri and Chapados, Nicolas and Bengio, Yoshua},
  journal = {arXiv preprint arXiv:1905.10437},
  year    = {2020},
  url     = {https://arxiv.org/pdf/2009.11961.pdf}
  :contentReference[oaicite:7]{index=7}
}

@article{bai2018tcn,
  title   = {An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling},
  author  = {Bai, Shaojie and Kolter, J Zico and Koltun, Vladlen},
  journal = {arXiv preprint arXiv:1803.01271},
  year    = {2018},
  url     = {https://arxiv.org/abs/1803.01271}
  :contentReference[oaicite:8]{index=8}
}

@article{kozodoi2024graphdeepar,
  title   = {Probabilistic Demand Forecasting with Graph Neural Networks},
  author  = {Kozodoi, Nikita and Zinovyeva, Elizaveta and Valentin, Simon and Pereira, João and Agundez, Rodrigo},
  journal = {arXiv preprint arXiv:2401.13096},
  year    = {2024},
  url     = {https://arxiv.org/abs/2401.13096}
  :contentReference[oaicite:9]{index=9}
}

@misc{m5-retail-transformers,
  title        = {Benchmarking Autoformer, Informer, and PatchTST on Retail M5 Forecasting},
  howpublished = {Experimento comparativo en serie de retail a gran escala},
  note         = {Mejora de hasta 29% en MASE sobre métodos clásicos}, 
  url          = {https://github.com}, 
  year         = {2023}
  
}

@misc{gatereport,
  title        = {Research on retail demand forecasting based on deep learning},
  author       = {{SPIE Digital Library}},
  year         = {2022},
  url          = {https://www.spiedigitallibrary.org/conference-proceedings-of-spie/12506/125066D/Research-on-retail-demand-forecasting-based-on-deep-learning/10.1117/12.2661780.pdf}
  :contentReference[oaicite:11]{index=11}
}
